In [1]:
import os
import sys
import json
import pandas
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [2]:
data = pandas.read_table(filepath_or_buffer="./pfm_train.csv", sep=",")
# a = pandas.get_dummies(data["Gender"])
# a.head()
def deal_age(data):
    if 18 <= int(data) < 25:
        return 1
    elif 25 <= int(data) < 30:
        return 2
    elif 30 <= int(data) < 40:
        return 3
    elif 40 <= int(data) < 50:
        return 4
    elif 50 <= int(data) < 60:
        return 5
    else:
        return 6
def deal_income(data):
    data = int(data)
    if data < 3000:
        return 1
    elif data < 5000:
        return 2
    elif data < 8000:
        return 3
    elif data < 10000:
        return 4
    elif data < 15000:
        return 5
    elif data < 20000:
        return 6
    elif data < 50000:
        return 7
    else:
        return 8
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,...,3,80,1,7,2,4,7,5,0,7
1,54,0,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,...,1,80,1,33,2,1,5,4,1,4
2,34,1,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,...,4,80,0,9,3,3,9,7,0,6
3,39,0,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,...,3,80,1,21,3,3,21,6,11,8
4,28,1,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,...,1,80,2,1,2,3,1,0,0,0


In [3]:
dummies_col = ["BusinessTravel", "Department", "EducationField", "JobRole", "Gender", "MaritalStatus", "Over18","OverTime"]
dum_l = []
for i in dummies_col:
    dum_l.append(pandas.get_dummies(data[i]))
l = [data] + dum_l
data = pandas.concat(l,axis=1)
data.drop(dummies_col, axis=1,inplace=True)
new_data = data.loc[:, ["Age", "MonthlyIncome", "PercentSalaryHike",
                                "PerformanceRating",
                                "RelationshipSatisfaction",
                                "StockOptionLevel",
                                "TotalWorkingYears",
                                "TrainingTimesLastYear",
                                "WorkLifeBalance",
                                "YearsAtCompany",
                                "YearsInCurrentRole","DistanceFromHome","YearsSinceLastPromotion","YearsWithCurrManager","NumCompaniesWorked",
                       "Education","EnvironmentSatisfaction","JobInvolvement","JobLevel","JobSatisfaction"]]
new_data = (new_data - new_data.mean()) / new_data.std()
data.drop(["Age", "MonthlyIncome", "EmployeeNumber", "PercentSalaryHike",
           "PerformanceRating",
           "RelationshipSatisfaction",
           "StandardHours",
           "StockOptionLevel",
           "TotalWorkingYears",
           "TrainingTimesLastYear",
           "WorkLifeBalance",
           "YearsAtCompany",
           "YearsInCurrentRole","DistanceFromHome","YearsSinceLastPromotion","YearsWithCurrManager","NumCompaniesWorked",
          "Education","EnvironmentSatisfaction","JobInvolvement","JobLevel","JobSatisfaction"], axis=1, inplace=True)
data = pandas.concat([data, new_data], axis=1)
# df1 = data[data["Attrition"]==1]
# df2 = data[data["Attrition"]==0].sample(frac=0.2)
# data=pandas.concat([df1,df2])
data.head()

,Attrition,Non-Travel,Travel_Frequently,Travel_Rarely,Human Resources,Research & Development,Sales,Human Resources,Life Sciences,Marketing,...,YearsInCurrentRole,DistanceFromHome,YearsSinceLastPromotion,YearsWithCurrManager,NumCompaniesWorked,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction
0,0,0,0,1,0,1,0,0,1,0,...,0.219100,-1.028131,-0.671840,0.799435,-0.670767,1.053833,-1.571376,-1.034746,-0.049237,0.240845
1,0,0,1,0,0,1,0,0,1,0,...,-0.057287,-1.028131,-0.370074,-0.034363,1.719655,1.053833,1.160732,0.380951,0.853449,0.240845
2,1,0,1,0,0,1,0,0,1,0,...,0.771874,-0.296128,-0.671840,0.521502,-0.670767,0.075591,-1.571376,-2.450443,-0.049237,0.240845
3,0,0,0,1,0,1,0,0,1,0,...,0.495487,-1.028131,2.647582,1.077367,-0.670767,-1.880892,1.160732,-1.034746,1.756135,1.141964
4,1,0,1,0,0,1,0,0,0,0,...,-1.162836,-1.028131,-0.671840,-1.146093,-0.670767,0.075591,-1.571376,-1.034746,-0.951924,-0.660275


In [4]:
y = data.Attrition
X = data.drop(["Attrition"], axis=1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2,random_state = 149)
pandas.Series(data["Attrition"]).value_counts()

0    922
1    178
Name: Attrition, dtype: int64

In [5]:
model = linear_model.LogisticRegression(solver='liblinear',C=1,penalty='l1', max_iter=100)
a = model.fit(X_train, y_train)
pred = model.predict(X=X_test)
# print(pandas.Series(pred).value_counts())
score = metrics.accuracy_score(y_test, pred)
score_true = metrics.recall_score(y_test, pred)
score_false = metrics.recall_score(y_test, pred, pos_label=0)
print("理论准确率：", model.score(X_train, y_train))
print("准确率:", score)
print("正例覆盖率：", score_true)
print("负例覆盖率：", score_false)

理论准确率： 0.9
准确率: 0.8863636363636364
正例覆盖率： 0.43243243243243246
负例覆盖率： 0.9781420765027322
